In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_columns', None)
from datetime import datetime
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/dnyaneshwarichaudhari/Documents/Jsonkey/fcdatalake-75264e184bf7.json"

from google.cloud import bigquery
import db_dtypes
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery_storage
credentials = service_account.Credentials.from_service_account_file("/home/dnyaneshwarichaudhari/Documents/Jsonkey/fcdatalake-75264e184bf7.json")
project_id = 'fcdatalake'
bqclient = bigquery.Client(credentials=credentials, project= project_id,)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)

In [2]:
Ecom_invoice = pd.read_csv("/home/dnyaneshwarichaudhari/Documents/consolidated_awb_excel_DIGITAL_AGE_RETAIL_PRIVATE_LIMITED_77219_2023_08_31.csv")
Region_master = pd.read_excel("/home/dnyaneshwarichaudhari/Documents/Master.xlsx")
zone_mapping = pd.read_excel("/home/dnyaneshwarichaudhari/Documents/zone_mapping.xlsx")

/tmp/ipykernel_114867/2398992987.py:1: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  Ecom_invoice = pd.read_csv("/home/dnyaneshwarichaudhari/Documents/consolidated_awb_excel_DIGITAL_AGE_RETAIL_PRIVATE_LIMITED_77219_2023_08_31.csv")


In [3]:
Region_master

,Ekart_zones,0.5_Grm,Additional_0.5_5 kg,Additional_1kg_(above 5)
0,Local,20,13,8
1,Regional,23,14,9
2,Metro,29,16,9
3,ROI,32,17,11
4,NE J&K,40,20,14


In [5]:
zone_mapping

,Billable Zone,Ekart_zones
0,Within Zones -ROS,Regional
1,Within Zones -UP,Regional
2,Intra-city,Local
3,Within Zones,Regional
4,Rest of India -ROS,ROI
5,NORTH EAST -ROS,NE J&K
6,Rest of India,ROI
7,Rest of India -UP,ROI
8,J&K -ROS,NE J&K
9,NORTH EAST -UP,NE J&K


In [6]:
list_airwaybill_number1 = tuple(map(str, Ecom_invoice['airwaybill_number'][0:53281]))
print(len(list_airwaybill_number1))
list_airwaybill_number2 = tuple(map(str, Ecom_invoice['airwaybill_number'][53281:106561]))
print(len(list_airwaybill_number2))
# list_airwaybill_number3 = tuple(map(str, Ecom_invoice['airwaybill_number'][100000:106561]))
# print(len(list_airwaybill_number3))

53281
53247


In [7]:
Q1 = '''SELECT
          shippingreferenceno AS airwaybill_number,
          shipmentlength,
          shipmentbreadth,
          shipmentheight,
          (shipmentlength*shipmentbreadth*shipmentheight)/5000 AS LBH,
          volweight,
          weightinkgs,
          shippingamount
        FROM
          `fcdatalake.FCDB.fc_shippingdetails` AS a 
        WHERE
          shippingreferenceno IN ('{}')
      '''.format("','".join(list_airwaybill_number1))

df1 = (
    bqclient.query(Q1)
    .result()
    .to_dataframe(
        create_bqstorage_client=True,
    )
)
print(df1.head(2))
print(df1.shape)

  airwaybill_number shipmentlength shipmentbreadth shipmentheight  \
0        9331950218   42.700000000    28.550000000   25.300000000   
1        9301059056   53.000000000    29.000000000   27.000000000   

           LBH    volweight  weightinkgs  shippingamount  
0  6.168570100  5.140000000  1.000000000   705.780000000  
1  8.299800000  6.920000000  3.000000000  1430.040000000  
(49697, 8)


In [8]:
Q2 = '''SELECT
          shippingreferenceno AS airwaybill_number,
          shipmentlength,
          shipmentbreadth,
          shipmentheight,
          (shipmentlength*shipmentbreadth*shipmentheight)/5000 AS LBH,
          volweight,
          weightinkgs,
          shippingamount
        FROM
          `fcdatalake.FCDB.fc_shippingdetails` AS a 
        WHERE
          shippingreferenceno IN ('{}')
      '''.format("','".join(list_airwaybill_number2))

df2 = (
    bqclient.query(Q2)
    .result()
    .to_dataframe(
        create_bqstorage_client=True,
    )
)
print(df2.head(2))
print(df2.shape)

  airwaybill_number shipmentlength shipmentbreadth shipmentheight  \
0        3026116034   43.000000000    38.000000000    8.000000000   
1        3099356150   26.000000000    11.000000000   39.000000000   

           LBH    volweight  weightinkgs shippingamount  
0  2.614400000  2.180000000  0.190000000  485.000000000  
1  2.230800000  1.860000000  1.000000000  955.170000000  
(49271, 8)


In [9]:
merged_df = pd.concat([df1, df2], ignore_index=True)
merged_df.shape

(98968, 8)

In [10]:
Ecom_invoice.airwaybill_number = Ecom_invoice.airwaybill_number.astype(int)
merged_df.airwaybill_number = merged_df.airwaybill_number.astype(int)

In [11]:
final_df =pd.merge(merged_df, Ecom_invoice, how='left',on='airwaybill_number')
final_df.shape

(98968, 56)

In [12]:
# final_df['chargable_weight'] = np.maximum(final_df['LBH'], final_df['weightinkgs'])
# final_df['chargable_weight']

In [13]:
final_df['Ekart_zones'] = final_df['Billable Zone / Rate'].map(zone_mapping.set_index('Billable Zone')['Ekart_zones'])

In [14]:
final_df =final_df.merge(Region_master, how='left',on='Ekart_zones')
final_df.shape

(98968, 60)

In [15]:
final_df['chargeable_weight']=final_df['chargeable_weight'].astype(float)

In [16]:
# final_df['chargable_weight']=np.round(final_df['chargable_weight'], 1)
# final_df['chargable_weight']

In [17]:
final_df['chargeable_weight']

0         5.5
1         7.0
2         2.5
3         3.0
4        34.0
         ... 
98963     4.0
98964     3.5
98965     3.5
98966     3.5
98967     4.0
Name: chargeable_weight, Length: 98968, dtype: float64

In [18]:
##Additional 1 kg (above 5)	for after every 1kg above 5 kg apply why divide 0.5
##total 5 kg or 0.5 + additional 5 kg

In [19]:
# Unit 1 is the units multiplier to cost of additional 0.5-5
# Unit 2 is the units multiplier to cost of additional 5+

final_df['Unit1'] = 0
final_df['Unit2'] = 0

# Iterate through rows
for index, row in final_df.iterrows():
    chargeable_weight = row['chargeable_weight']
    if chargeable_weight < 0.5:
        final_df.loc[index, 'Unit1'] = 0
    elif 0.5 <= chargeable_weight <= 5:
        final_df.loc[index, 'Unit1'] = (chargeable_weight-0.5)/0.5
    elif chargeable_weight>5:
        final_df.loc[index, 'Unit1'] =  (5 - 0.5) / 0.5
        final_df.loc[index, 'Unit2'] =  (chargeable_weight - 5) / 1

In [20]:
final_df['Unit1'] = np.ceil(final_df['Unit1']) #rounding up the values for >5
final_df['Unit2'] = np.ceil(final_df['Unit2'])

In [21]:
final_df['shippingamount'] = final_df['shippingamount'].astype('float')

In [22]:
# final_df['COD_charges1']=final_df['shippingamount']*0.1
final_df['COD_charges']= 10
final_df.shape
# final_df['COD_charges'] = np.maximum(final_df['COD_charges1'], final_df['COD_charges2'])

(98968, 63)

In [23]:
final_df['Status'].dtype

dtype('O')

In [24]:
COD=final_df[final_df.product_type=='cod']
PPD=final_df[final_df.product_type=='ppd']
PPD['COD_charges']=0

/tmp/ipykernel_103358/4159415774.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PPD['COD_charges']=0


In [25]:
COD['freight']=COD['0.5_Grm']+(COD['Unit1']*COD['Additional_0.5_5 kg'])+(COD['Additional_1kg_(above 5)']*COD['Unit2'])+COD['COD_charges']
COD['GST']=COD['freight']*0.18
COD['Forword_freight']=COD['freight']+COD['GST']
COD

/tmp/ipykernel_103358/3654942616.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  COD['freight']=COD['0.5_Grm']+(COD['Unit1']*COD['Additional_0.5_5 kg'])+(COD['Additional_1kg_(above 5)']*COD['Unit2'])+COD['COD_charges']
/tmp/ipykernel_103358/3654942616.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  COD['GST']=COD['freight']*0.18
/tmp/ipykernel_103358/3654942616.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

,airwaybill_number,shipmentlength,shipmentbreadth,shipmentheight,LBH,volweight,weightinkgs,shippingamount,Shipper,order_number,date,time,product_type,Origin Pincode,Destination Pincode,Origin,Destination,chargeable_weight,declared_value,collectable_value,Freight,COD,sdl_charge,reverse_charge,sdd_charge,vchc,First_Mile,RTO_Charge,fuel_surcharge,QC_CHARGE,NDD_CHARGE,SDD_CHARGE,CONVENIENCE_FEE,TRANSACTION_FEE,COVID_SURCHARGE,HANDLING_CHARGES,BBD_CHARGES,OTP_CHARGES,CPL_CHARGES,CDL_CHARGES,FOV_CHARGES,Total,Card_Type,Status,Parent/RTS AWB,Parent/RTS AWB Date,rts_status,rts_status.1,DG_Flag,Billable Zone / Rate,ecomgst_state,customer gst_state,Ecom GST,Customer GST,Invoice No,Invoice Date,Ekart_zones,0.5_Grm,Additional_0.5_5 kg,Additional_1kg_(above 5),Unit1,Unit2,COD_charges,freight,GST,Forword_freight
0,9331950218,42.700000000,28.550000000,25.300000000,6.168570100,5.140000000,1.000000000,705.78,8323481387454-PNQ,9246720ZCDF28C703,07-08-2023,8:45:00,cod,410501,122103,PUNE - PNQ,NUH,5.5,1295.0,705.78,227,25.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,252.0,NaN,Delivered,NaN,NaN,0,Pickup,1.0,Rest of India -ROS,MH,MH,27AADCE1344F1Z0,27AADCD8136E2ZQ,2.72E+14,31-08-2023,ROI,32,17,11,9.0,1.0,10,206.0,37.08,243.08
1,9301059056,53.000000000,29.000000000,27.000000000,8.299800000,6.920000000,3.000000000,1430.04,8323482683511-BLR,23555185ARH18A9890,02-08-2023,12:42:00,cod,562114,682504,HOSAKOTE ROS,KOCHI - KCI,7.0,1795.0,1430.04,213,25.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,248.0,NaN,Undelivered,1.002220e+10,14-08-2023 15:41,2,Pickup,1.0,Within Zones -ROS,KA,MH,29AADCE1344F1ZW,27AADCD8136E2ZQ,292000000000000.0,31-08-2023,Regional,23,14,9,9.0,2.0,10,177.0,31.86,208.86
7,9100371441,20.000000000,17.000000000,44.000000000,2.992000000,2.490000000,0.790000000,327.16,8323481713777-M/s Chaya Traders (J P Nagar - B...,15504113DJGB68DF5F,17-08-2023,12:19:00,cod,560062,524121,BENGALURU - BLR,SULLURPET ROS,2.5,399.0,327.16,93,25.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,118.0,SAVINGS,Delivered,NaN,NaN,0,Pickup,1.0,Within Zones -ROS,KA,MH,29AADCE1344F1ZW,27AADCD8136E2ZQ,292000000000000.0,31-08-2023,Regional,23,14,9,4.0,0.0,10,89.0,16.02,105.02
8,9301146417,30.480000000,27.940000000,33.530000000,5.710904707,4.760000000,4.000000000,1328.67,8323481713777-M/s Chaya Traders (J P Nagar - B...,21238789CEI643BA97,13-08-2023,12:32:00,cod,560062,577174,BENGALURU - BLR,CHICKAMAGALUR ROS,5.0,1998.0,1328.67,153,25.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,188.0,NaN,Undelivered,1.002255e+10,15-08-2023 19:12,2,Pickup,1.0,Within Zones -ROS,KA,MH,29AADCE1344F1ZW,27AADCD8136E2ZQ,292000000000000.0,31-08-2023,Regional,23,14,9,9.0,0.0,10,159.0,28.62,187.62
10,9332028102,18.000000000,23.000000000,13.000000000,1.076400000,0.900000000,0.680000000,260.20,8323481713777-M/s Chaya Traders (J P Nagar - B...,11397423PVH31AE58D,29-08-2023,12:24:00,cod,560062,689649,BENGALURU - BLR,PATHANAMTHITTA ROS,1.5,365.0,260.20,63,25.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,98.0,NaN,Delivered,NaN,NaN,0,Pickup,1.0,Within Zones -ROS,KA,MH,29AADCE1344F1ZW,27AADCD8136E2ZQ,2.92E+14,31-08-2023,Regional,23,14,9,2.0,0.0,10,61.0,10.98,71.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98956,9300828365,34.000000000,41.000000000,13.000000000,3.624400000,3.020000000,2.000000000,716.07,8323482522696-Min2Max Retail Company (Guindy -...,24035930HOR8E9A48A,24-08-2023,13:35:00,cod,600056,643219,CHENNAI - MAA,OOTY - OTY,3.5,1299.0,716.07,112,25.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,147.0,NaN,Delivered,NaN,NaN,0,Pickup,1.0,Within Zones -UP,TN,MH,33AADCE1344F1Z7,27AADCD8136E2ZQ,332000000000000.0,31-08-2023,Regional,23,14,9,6.0,0.0,10,117.0,21.06,138.06
98958,8715232941,34.000000000,41.000000000,13.000000000,3.624400000,3.020000000,1.000000000,795.17,8323482937458-S. S. Retail Agency (Begumpet - ...,

In [26]:
PPD['freight']=PPD['0.5_Grm']+(PPD['Unit1']*PPD['Additional_0.5_5 kg'])+(PPD['Additional_1kg_(above 5)']*PPD['Unit2'])+PPD['COD_charges']
PPD['GST']=PPD['freight']*0.18
PPD['Forword_freight']=PPD['freight']+PPD['GST']

/tmp/ipykernel_103358/3528086751.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PPD['freight']=PPD['0.5_Grm']+(PPD['Unit1']*PPD['Additional_0.5_5 kg'])+(PPD['Additional_1kg_(above 5)']*PPD['Unit2'])+PPD['COD_charges']
/tmp/ipykernel_103358/3528086751.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PPD['GST']=PPD['freight']*0.18
/tmp/ipykernel_103358/3528086751.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [27]:
COD_PPD=pd.concat([COD,PPD],sort=False)

In [28]:
COD_PPD['freight1'] = COD_PPD['Forword_freight']-10

In [29]:
COD_PPD['RTO_fright']= COD_PPD[(COD_PPD['Status'] =='Undelivered') & (COD_PPD['Parent/RTS AWB'].isna())]['freight1']*0.70

In [30]:
COD_PPD.loc[COD_PPD['RTO_fright'].notna(), 'Forword_freight'] = 0

In [31]:
COD_PPD['RTO_fright']=COD_PPD['RTO_fright'].fillna(0)

In [32]:
COD_PPD['Ecom_Freight'] = COD_PPD['Forword_freight'].values+COD_PPD['RTO_fright'].values

In [33]:
COD_PPD

,airwaybill_number,shipmentlength,shipmentbreadth,shipmentheight,LBH,volweight,weightinkgs,shippingamount,Shipper,order_number,date,time,product_type,Origin Pincode,Destination Pincode,Origin,Destination,chargeable_weight,declared_value,collectable_value,Freight,COD,sdl_charge,reverse_charge,sdd_charge,vchc,First_Mile,RTO_Charge,fuel_surcharge,QC_CHARGE,NDD_CHARGE,SDD_CHARGE,CONVENIENCE_FEE,TRANSACTION_FEE,COVID_SURCHARGE,HANDLING_CHARGES,BBD_CHARGES,OTP_CHARGES,CPL_CHARGES,CDL_CHARGES,FOV_CHARGES,Total,Card_Type,Status,Parent/RTS AWB,Parent/RTS AWB Date,rts_status,rts_status.1,DG_Flag,Billable Zone / Rate,ecomgst_state,customer gst_state,Ecom GST,Customer GST,Invoice No,Invoice Date,Ekart_zones,0.5_Grm,Additional_0.5_5 kg,Additional_1kg_(above 5),Unit1,Unit2,COD_charges,freight,GST,Forword_freight,freight1,RTO_fright,Ecom_Freight
0,9331950218,42.700000000,28.550000000,25.300000000,6.168570100,5.140000000,1.000000000,705.78,8323481387454-PNQ,9246720ZCDF28C703,07-08-2023,8:45:00,cod,410501,122103,PUNE - PNQ,NUH,5.5,1295.0,705.78,227,25.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,252.0,NaN,Delivered,NaN,NaN,0,Pickup,1.0,Rest of India -ROS,MH,MH,27AADCE1344F1Z0,27AADCD8136E2ZQ,2.72E+14,31-08-2023,ROI,32,17,11,9.0,1.0,10,206.0,37.08,243.08,233.08,0.0,243.08
1,9301059056,53.000000000,29.000000000,27.000000000,8.299800000,6.920000000,3.000000000,1430.04,8323482683511-BLR,23555185ARH18A9890,02-08-2023,12:42:00,cod,562114,682504,HOSAKOTE ROS,KOCHI - KCI,7.0,1795.0,1430.04,213,25.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,248.0,NaN,Undelivered,1.002220e+10,14-08-2023 15:41,2,Pickup,1.0,Within Zones -ROS,KA,MH,29AADCE1344F1ZW,27AADCD8136E2ZQ,292000000000000.0,31-08-2023,Regional,23,14,9,9.0,2.0,10,177.0,31.86,208.86,198.86,0.0,208.86
7,9100371441,20.000000000,17.000000000,44.000000000,2.992000000,2.490000000,0.790000000,327.16,8323481713777-M/s Chaya Traders (J P Nagar - B...,15504113DJGB68DF5F,17-08-2023,12:19:00,cod,560062,524121,BENGALURU - BLR,SULLURPET ROS,2.5,399.0,327.16,93,25.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,118.0,SAVINGS,Delivered,NaN,NaN,0,Pickup,1.0,Within Zones -ROS,KA,MH,29AADCE1344F1ZW,27AADCD8136E2ZQ,292000000000000.0,31-08-2023,Regional,23,14,9,4.0,0.0,10,89.0,16.02,105.02,95.02,0.0,105.02
8,9301146417,30.480000000,27.940000000,33.530000000,5.710904707,4.760000000,4.000000000,1328.67,8323481713777-M/s Chaya Traders (J P Nagar - B...,21238789CEI643BA97,13-08-2023,12:32:00,cod,560062,577174,BENGALURU - BLR,CHICKAMAGALUR ROS,5.0,1998.0,1328.67,153,25.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,188.0,NaN,Undelivered,1.002255e+10,15-08-2023 19:12,2,Pickup,1.0,Within Zones -ROS,KA,MH,29AADCE1344F1ZW,27AADCD8136E2ZQ,292000000000000.0,31-08-2023,Regional,23,14,9,9.0,0.0,10,159.0,28.62,187.62,177.62,0.0,187.62
10,9332028102,18.000000000,23.000000000,13.000000000,1.076400000,0.900000000,0.680000000,260.20,8323481713777-M/s Chaya Traders (J P Nagar - B...,11397423PVH31AE58D,29-08-2023,12:24:00,cod,560062,689649,BENGALURU - BLR,PATHANAMTHITTA ROS,1.5,365.0,260.20,63,25.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,98.0,NaN,Delivered,NaN,NaN,0,Pickup,1.0,Within Zones -ROS,KA,MH,29AADCE1344F1ZW,27AADCD8136E2ZQ,2.92E+14,31-08-2023,Regional,23,14,9,2.0,0.0,10,61.0,10.98,71.98,61.98,0.0,71.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98960,3026138746,34.000000000,41.000000000,13.000000000,3.624400000,3.020000000,2.000000000,654.70,8323482522696-Min2Max Retail Company (Guindy -...,12393815TYZDF95767,07-08-2023,12:40:00,ppd,600056,502334,CHENNAI - MAA,NARSAPUR TG,3.5,1299.0,0.00,123,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123.0,NaN,Delivered,NaN,NaN,0,Pickup,1.0,Within Zones -ROS,TN,MH,33AADCE1344F1Z7,27AADCD8136E2ZQ,332000000000000.0,31-08-2023,Regional,23,14,9,6.0,0.0,0,107.0,19.26,126.26,116.26,0.0,126.26
98

In [34]:
COD_PPD.to_csv("/home/dnyaneshwarichaudhari/Documents/ATS_Freight.csv")